In [9]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO
import glob

In [46]:
def job_date(df):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(str(df.iloc[0,0]).split(' ')[0], errors='coerce')
    return job, edit_date



In [3]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
c_recents = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\c_recent\\"
on_going_path = ''

In [4]:
test_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections\Projection 10378 Brunner Farm-CGRS 20231128.xlsx"

In [5]:
files = []
for item in os.listdir(completed_path):
    job_code = item.split(' ')[1]
    files.append(job_code)
files= pd.unique(files)
    
#     full_path = os.path.join(completed_path, item)

In [18]:
len(latest_file_list)

66

In [53]:
latest_file_list = []
for file in files:
    job_files = [os.path.join(completed_path, x) for x in os.listdir(completed_path) if file in x]
    latest_file = max(job_files, key=os.path.getctime)
    latest_file_list.append(latest_file)
latest_file_list = pd.unique(latest_file_list)   

In [57]:
df_list = []
for item in latest_file_list:
    num_sheet = len(pd.ExcelFile(item).sheet_names)
    for sheet in range(0,num_sheet):
        xl = pd.read_excel(item ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        job, date = job_date(df)
        df.columns = df.iloc[2]
        df = df.drop(df.index[:3])
        df['updated'] = date
        df['job'] = job
        df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
#         print(job)
#         print(job)
        #         print(date)
#         df = df.reset_index()
        df_list.append(df)
# full_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True)
        #         print(df.head(3))

In [55]:
latest_file_list

array(['C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10291-30-50 Stockyard SYEC NWC - A&P 20220613.xlsx',
       'C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10291-50 Stockyard SYEC NWC - A&P 20220601.xlsx',
       'C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10292-50 Sorrel Ranch-Lokal 20220616.xlsx',
       'C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10293-50 CU FYSH - JE Dunn 20220616.xlsx',
       "C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10298-30-50 Alta Sloan's Lake-Wood Partners 20220810.xlsx",
       "C:\\Users\\willd\\OneDrive\\Desktop\\Belair\\projection_analysis\\completed proj projections\\Projection 10298-50 Alta Sloan's Lake-Wood Partners 20220627.xlsx",
       'C:\\Use

## handle the 2 sheet 30-50 issues

In [81]:
xl = pd.read_excel(test_path)
df = pd.DataFrame(xl)
job, date = job_date(df)
df.columns = df.iloc[2]
df = df.drop(df.index[:3])
df['updated'] = date
df['job'] = job
# remove non numeric values in this column
df = df[df['Phase Code'].apply(lambda x: str(x).isnumeric())]
df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)

In [20]:
df_list = []
for item in os.listdir(completed_path):
    full_path = os.path.join(completed_path, item)
    num_sheet = len(pd.ExcelFile(full_path).sheet_names)
    for sheet in range(0,num_sheet):
        xl = pd.read_excel(full_path ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        df.columns.values[0] = "1"
        df['1'] = df['1'].str.strip()
#         if 'Phase Code' in df.iloc[:,0]: 
        print(df.head(5))

                           1         Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  6/29/22 Erica, Steve, Jen                NaN        NaN        NaN   
1                        NaN                NaN        NaN        NaN   
2                 Phase Code        Description      Notes        Qty   
3                        NaN        Contingency        NaN          1   
4                        NaN  Field Supervision        NaN         18   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                 7020.74   
4           15        WKS   0.833333         NaN                 9892.29   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...           

                          1         Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  5/5/22 Erica, Steve, Jen                NaN        NaN        NaN   
1                       NaN                NaN        NaN        NaN   
2                Phase Code        Description      Notes        Qty   
3                       NaN        Contingency        NaN          1   
4                       NaN  Field Supervision        NaN         18   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                13172.19   
4           12        WKS   0.666667         NaN                 10463.8   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...              NaN

                                  1         Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  2/23/22 Erica, Steve, Tracy, Jen                NaN        NaN        NaN   
1                               NaN                NaN        NaN        NaN   
2                        Phase Code        Description      Notes        Qty   
3                               NaN        Contingency        NaN          1   
4                               NaN  Field Supervision        NaN         18   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                13172.19   
4           11        WKS   0.611111         NaN                 10463.8   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unn

                                                1  \
0  1/10/22 Kai, Hutto, Brandon, Tracy, Steve, Jen   
1                                             NaN   
2                                      Phase Code   
3                                             NaN   
4                                             NaN   

                     Unnamed: 1 Unnamed: 2 Unnamed: 3   Unnamed: 4 Unnamed: 5  \
0                           NaN        NaN        NaN          NaN        NaN   
1                           NaN        NaN        NaN          NaN        NaN   
2                   Description      Notes        Qty  HJ Qty Comp       Unit   
3                   Contingency        NaN          1          NaN         LS   
4  Travel/Per Diem/Field Office        NaN          1          NaN         LS   

  Unnamed: 6  Unnamed: 7              Unnamed: 8   Unnamed: 9  ...  \
0        NaN         NaN                     NaN          NaN  ...   
1        NaN         NaN                     NaN      

                  1                    Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  4/19/22 Kai, Jen                           NaN        NaN        NaN   
1               NaN                           NaN        NaN        NaN   
2        Phase Code                   Description      Notes        Qty   
3               NaN                   Contingency        NaN          1   
4               NaN  Travel/Per Diem/Field Office        NaN          1   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                19477.84   
4          NaN         LS          0         NaN                    2400   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ..

...

                               1   Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  6/6/22 Kenny, Kai, Steve, Jen          NaN        NaN        NaN   
1                            NaN          NaN        NaN        NaN   
2                     Phase Code  Description      Notes        Qty   
3                            NaN  Incidentals        NaN          1   
4                            NaN  Contingency        NaN          1   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                       0   
4          NaN         LS          0         NaN                14875.53   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...              NaN      

                                1   Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  7/18/22 Kenny, Kai, Steve, Jen          NaN        NaN        NaN   
1                             NaN          NaN        NaN        NaN   
2                      Phase Code  Description      Notes        Qty   
3                             NaN  Incidentals        NaN          1   
4                             NaN  Contingency        NaN          1   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                       0   
4          NaN         LS          0         NaN                14875.53   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...              NaN

                                1   Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  8/16/22 Kenny, Kai, Steve, Jen          NaN        NaN        NaN   
1                             NaN          NaN        NaN        NaN   
2                      Phase Code  Description      Notes        Qty   
3                             NaN  Incidentals        NaN          1   
4                             NaN  Contingency        NaN          1   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                       0   
4          NaN         LS          0         NaN                15931.81   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...              NaN

                                 1   Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  10/17/22 Kenny, Kai, Steve, Jen          NaN        NaN        NaN   
1                              NaN          NaN        NaN        NaN   
2                       Phase Code  Description      Notes        Qty   
3                              NaN  Incidentals        NaN          1   
4                              NaN  Contingency        NaN          1   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN         LS          0         NaN                       0   
4          NaN         LS          0         NaN                15931.81   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...           

                  1              Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  Date & Attendees                     NaN        NaN        NaN   
1               NaN                     NaN        NaN        NaN   
2        Phase Code             Description      Notes        Qty   
3               NaN  Unproductive Equipment        NaN        NaN   
4               NaN         Rework/Warranty        NaN        NaN   

    Unnamed: 4 Unnamed: 5 Unnamed: 6  Unnamed: 7              Unnamed: 8  \
0          NaN        NaN        NaN         NaN                     NaN   
1          NaN        NaN        NaN         NaN                     NaN   
2  HJ Qty Comp       Unit     % Comp         NaN  Budget Last Projection   
3          NaN        HRS        NaN         NaN                     NaN   
4          NaN        HRS        NaN         NaN                     NaN   

    Unnamed: 9  ...      Unnamed: 22 Unnamed: 23           Unnamed: 24  \
0          NaN  ...              NaN         NaN      

In [40]:
df_list = []
for item in os.listdir(completed_path):
    full_path = os.path.join(completed_path, item)
    num_sheet = len(pd.ExcelFile(full_path).sheet_names)
    for sheet in range(0,num_sheet):
        
        xl = pd.read_excel(full_path ,sheet_name=sheet)
        df = pd.DataFrame(xl)
        df.columns.values[0] = "1"
#         df['1'] = df['1'].fillna(False)
        pc_list = df.index[df['1'].str.contains('Phase Code', na= False)].tolist()
        if len(pc_list) > 0:
            pc_ind = pc_list[0]
#         if 'Phase Code' in df.iloc[:,0]:   
#             job, date = job_date(df)
#             df.columns = df.iloc[pc_ind]
#             df = df.drop(df.index[:3])
#             df['updated'] = date
#             df['job'] = job
#             # remove non numeric values in this column
#             df = df[df['Phase Code'].apply(lambda x: str(x).isnumeric())]
#             df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
            df_list.append(df)
full_df = pd.concat(df_list)    



In [37]:
pc_list = df.index[df['1'].str.contains('Phase Code', na= False)]
pc_list[0]

2

In [41]:
full_df

,1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,"6/29/22 Erica, Steve, Jen",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Over/ (Under) Budget,% Comp,NaN,Over/ (Under) Variance,NaN,NaN,NaN,NaN,NaN,NaN
3,1001,Contingency,NaN,1,NaN,LS,0,NaN,7020.74,NaN,...,-1129.7,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1010,Field Supervision,NaN,18,15,WKS,0.833333,NaN,9892.29,NaN,...,2213.98,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Totals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16261.83,0,...,-8052.11,0.736045,NaN,-8052.11,NaN,NaN,NaN,NaN,NaN,NaN
25,Source Data Totals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16262,NaN,...,NaN,NaN,NaN,-8052.11,NaN,NaN,NaN,NaN,NaN,NaN
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
full_path

In [10]:
if 'Phase Code' in df.iloc[:,0]:
    print(1)

In [15]:
df.iloc[:,0] = df.iloc[:,0].str.strip()

In [21]:
df.iloc[:, 0]

0     11/28/23 Scotty, Steve, Trent, Kris, Andrew, A...
1                                                   NaN
2                                            Phase Code
3                                                   NaN
4                                                   NaN
5                                                   NaN
6                                                   NaN
7                                                   NaN
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16                                                  NaN
17                                              

In [28]:
print(job)
print(date)

10378 Brunner Farm-CGRS
2023-11-28 00:00:00


In [22]:
df.head(5)

,1,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,"11/28/23 Scotty, Steve, Trent, Kris, Andrew, A...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1st Projection 11/28/23,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,Budget Remaining,Over/ (Under) Budget,% Comp,NaN,Over/ (Under) Variance
3,NaN,Incidentals,NaN,0,NaN,LS,NaN,NaN,0,NaN,...,0,NaN,0,NaN,0,0,0,1,NaN,0
4,NaN,Contingency,NaN,1,NaN,LS,0,NaN,111.99,NaN,...,0,NaN,0,NaN,0,0,-111.99,1,NaN,-111.99


In [63]:
len(df.columns)

32